Data load and YOLOv5 cloning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies

import torch
import os
import cv2
import numpy
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from IPython.display import Image, clear_output  # to display images
from PIL import Image
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-242-ga80dd66 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/78.2 GB disk)


Training the model

In [ ]:
# Train YOLOv5s
!python train.py --img 416 --batch 64 --epochs 150 --data /content/drive/MyDrive/datasets/face_recognition/face_recognition.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/datasets/face_recognition/face_recognition.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=64, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-236-gdcf8073 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0

Model validation on the test set

In [ ]:
!python val.py --weights runs/train/exp/weights/best.pt --data /content/drive/MyDrive/datasets/face_recognition/face_recognition.yaml --img 416 --iou 0.65 --task test

val: data=/content/drive/MyDrive/datasets/face_recognition/face_recognition.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.65, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-236-gdcf8073 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/drive/MyDrive/datasets/face_recognition/labels/test.cache' images and labels... 270 found, 0 missing, 0 empty, 0 corrupt: 100% 270/270 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 9/9 [00:06<00:00,  1.29it/s]
                 all        270        692      0.868       0.75      0.829      0.464
Speed: 0.1ms pre-process, 2.9ms inf

Here we use detect to see how the model works with finding faces

In [ ]:
!python detect.py --weights /content/best.pt --img 416 --conf 0.4 --source /content/drive/MyDrive/datasets/face_recognition/images/test

Code in the blocks below is used to get images, find the coordinates of bounding boxes and blur that part of the image. 

In [ ]:
model = torch.hub.load('/content/yolov5/', 'custom', path='/content/best.pt', source='local')  # local repo
folder = '/content/drive/MyDrive/datasets/face_recognition/images/test/'

imgs = []
for filename in os.listdir(folder):
    print(filename[0:3])
    img = Image.open(folder+filename)
    imgs.append(img)


results = model(imgs, size=416)

In [ ]:
def pixelize(img, N):
    img2 = cv2.resize(img, 
                      (int(img.shape[1] / N), int(img.shape[0] / N)), 
                      interpolation = cv2.INTER_NEAREST)
    
    img3 = cv2.resize(img2, 
                      (img.shape[1], img.shape[0]), 
                      interpolation = cv2.INTER_NEAREST)
    
    return img3

In [ ]:
dir = '/content/image_results'
os.chdir(dir)
for i, img in enumerate(imgs):
  filename = img.filename
  img = img.convert('RGB') 
  open_cv_image = numpy.array(img) 
  # Convert RGB to BGR 
  open_cv_image = open_cv_image[:, :, ::-1].copy() 

  df_results = results.pandas().xyxy[i]  # im1 predictions (pandas)
  for index, row in df_results.iterrows():
    #print(row['xmin'], row['ymin'], row['xmax'], row['ymax'])
    try:
      img_p = pixelize(open_cv_image[int(row['ymin']):int(row['ymax']),
                                   int(row['xmin']):int(row['xmax'])], 3)
    except:
      print('greska')
      break
    open_cv_image[int(row['ymin']):int(row['ymax']),
                  int(row['xmin']):int(row['xmax'])] = img_p
  
  cv2.imwrite(filename.split('/')[-1], open_cv_image)